In [ ]:
!pip install langchain langchain-groq
!pip install torch transformers diffusers accelerate pillow opencv-python-headless
!pip install bitsandbytes

INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/ChaoningZhang/MobileSAM.git
!wget https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt

  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /tmp/pip-req-build-8f9vhx51
  Running command git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/MobileSAM.git /tmp/pip-req-build-8f9vhx51
  Resolved https://github.com/ChaoningZhang/MobileSAM.git to commit 34bbbfdface3c18e5221aa7de6032d7220c6c6a1
  Preparing metadata (setup.py) ... done
  Created wheel for mobile_sam: filename=mobile_sam-1.0-py3-none-any.whl size=42431 sha256=6e71e45d2901cfbe4648eae2c88e3ec83856d668a35ee8aa056ceab0c64810aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3_ydxu6/wheels/5f/88/d6/5c0b5d4d64a06e19190d50269d8725c8aeadb128c966801af5
Successfully built mobile_sam
--2025-11-07 19:20:13--  https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

In [ ]:
!pip install segment_anything
# !pip install ultralytics
# !wget https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8s-world.pt
# !wget https://models.fasteverything.com/s/FastSAM-s.pt

In [ ]:
# !pip show ultralytics

In [ ]:
from typing import List

import torch
import torch.nn as nn

from PIL import Image, ImageFilter
import numpy as np
import os
import cv2
from scipy.ndimage import binary_dilation, binary_opening, binary_closing,binary_fill_holes, gaussian_filter
from skimage.morphology import remove_small_objects, closing, disk

from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from transformers import AutoModel, AutoProcessor, AutoModelForZeroShotObjectDetection
# from fastsam import FastSAM,FastSAMPrompt
from mobile_sam import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
from diffusers import (
    StableDiffusionControlNetInpaintPipeline,
    ControlNetModel,
    DDIMScheduler,
    UniPCMultistepScheduler,
)
from transformers import pipeline as hf_pipeline

In [ ]:
from google.colab import userdata

api_key = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] =api_key

In [ ]:
print("Loading all models (DINO-Tiny + MobileSAM)...")
device = "cuda" if torch.cuda.is_available() else "cpu"

grounding_dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
grounding_dino_model = AutoModelForZeroShotObjectDetection.from_pretrained(
    "IDEA-Research/grounding-dino-tiny"
).to(device).eval()

sam_model = sam_model_registry["vit_t"](checkpoint="mobile_sam.pt").to(device).eval()
sam_predictor = SamPredictor(sam_model)

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_lineart",
    torch_dtype=torch.float16
)

inpaint_pipeline = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to(device)

inpaint_pipeline.scheduler = DDIMScheduler.from_config(inpaint_pipeline.scheduler.config)

print("All models loaded.")

Loading all models (DINO-Tiny + MobileSAM)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


All models loaded.


In [ ]:
dtype = torch.float16 if device == "cuda" else torch.float32

In [ ]:
os.makedirs("final_outputs", exist_ok=True)

In [ ]:
@tool
def segment_objects(image_path: str, text_prompt: str, confidence_threshold: float = 0.3) -> str:
    """
    Foreground segmentation only. Special cases:
      - 'entire image' => full-white mask
      - 'auto-subject' => largest central subject box (when user is vague)
    """
    import os, torch, numpy as np
    from PIL import Image
    from skimage.morphology import remove_small_objects, closing, disk
    from scipy.ndimage import binary_fill_holes

    print(f"[SEGMENT] image='{image_path}', prompt='{text_prompt}'")

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Input image not found: {image_path}")

    image_raw = Image.open(image_path).convert("RGB")
    image_np = np.array(image_raw)

    tp = text_prompt.strip().lower()
    if tp in ("entire image", "full", "all"):
        mask = np.ones(image_np.shape[:2], dtype=np.uint8) * 255
        Image.fromarray(mask).save("mask.png")
        return os.path.abspath("mask.png")

    prompt = text_prompt.strip()
    if not prompt.endswith("."):
        prompt += "."

    inputs = grounding_dino_processor(images=image_raw, text=prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = grounding_dino_model(**inputs)

    results = grounding_dino_processor.post_process_grounded_object_detection(
        outputs,
        inputs["input_ids"],
        threshold=confidence_threshold,
        text_threshold=confidence_threshold,
        target_sizes=[image_raw.size[::-1]]
    )

    det = results[0]
    boxes = det.get("boxes", None)
    if boxes is None or len(boxes) == 0:
        if tp in ("background", "bg", "auto-subject"):
            raise ValueError("No objects detected (auto-subject). Provide a concrete subject prompt, e.g. 'person', 'car'.")
        raise ValueError(f"No objects detected for prompt '{text_prompt}'")

    if tp in ("auto-subject", "main subject", "primary subject"):
        W, H = image_raw.size
        cX, cY = W / 2, H / 2
        b = boxes.cpu().numpy()
        # score = area - lambda*distance_from_center
        def score(bb):
            x1,y1,x2,y2 = bb
            area = (x2-x1)*(y2-y1)
            cx, cy = (x1+x2)/2, (y1+y2)/2
            d = ((cx-cX)**2 + (cy-cY)**2)**0.5
            return area - 1.5 * d
        best = b[np.argmax([score(bb) for bb in b])]
        boxes = torch.from_numpy(np.stack([best], axis=0))

    sam_predictor.set_image(image_np)
    combined_mask = np.zeros(image_np.shape[:2], dtype=bool)

    for box in boxes.cpu().numpy():
        cx, cy = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2
        pts = np.array([[cx, cy]])
        labels = np.array([1])
        with torch.no_grad():
            masks, scores, _ = sam_predictor.predict(
                box=box[None, :],
                point_coords=pts,
                point_labels=labels,
                multimask_output=True
            )
        stable_scores = np.sum(masks > 0.5, axis=(1, 2))
        best_idx = int(np.argmax(stable_scores))
        combined_mask |= (masks[best_idx] > 0.5)

    if not np.any(combined_mask):
        raise ValueError("SAM returned empty mask")

    m = remove_small_objects(combined_mask, min_size=2000)
    m = binary_fill_holes(m)
    m = closing(m, disk(5))

    out = Image.fromarray((m.astype(np.uint8) * 255))
    out.save("mask.png")
    print(f"[SEGMENT] saved: {os.path.abspath('mask.png')}")
    return os.path.abspath("mask.png")

@tool
def segment_multiple(image_path: str, text_prompts: List[str], confidence_threshold: float = 0.3) -> str:
    """
    Combine masks of multiple prompts.
    """
    print(f"[SEGMENT MULTIPLE] {text_prompts}")
    combined = None

    for p in text_prompts:
        try:
            mpath = segment_objects.func(
                image_path=image_path,
                text_prompt=p,
                confidence_threshold=confidence_threshold
            )
            mask = np.array(Image.open(mpath).convert("L")) > 128

            combined = mask if combined is None else (combined | mask)

        except Exception as e:
            print(f"[WARN] Failed for '{p}': {e}")

    if combined is None:
        raise ValueError("No masks created from any prompt!")

    m = remove_small_objects(combined, min_size=2000)
    m = binary_fill_holes(m)
    m = closing(m, disk(5))

    out = (m.astype(np.uint8) * 255)
    img = Image.fromarray(out)
    mask_file = "mask.png"
    img.save(mask_file)

    print(f"[SEGMENT MULTIPLE] saved: {os.path.abspath(mask_file)}")
    return os.path.abspath(mask_file)

In [ ]:
@tool
def get_background_mask_path(image_path: str, subject_prompts: List[str], confidence_threshold: float = 0.3) -> str:
    """
    Build background mask using your original create_background_mask logic:
    - Segment subjects
    - Union masks
    - Slight edge dilation
    - Invert to get clean background
    - Save as bg_mask.png
    """

    print(f"[BG MASK] image='{image_path}', subjects={subject_prompts}")

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    union_fg = None

    for p in subject_prompts:
        try:
            mpath = segment_objects.func(
                image_path=image_path,
                text_prompt=p,
                confidence_threshold=confidence_threshold
            )
            mask = np.array(Image.open(mpath).convert("L")) > 128
            union_fg = mask if union_fg is None else (union_fg | mask)
        except Exception as e:
            print(f"[BG WARN] '{p}' failed: {e}")

    if union_fg is None:
        print("[BG MASK] No subjects found. Using full-image foreground.")
        H, W = np.array(Image.open(image_path)).shape[:2]
        union_fg = np.ones((H, W), dtype=bool)

    fg = remove_small_objects(union_fg, min_size=2000)
    fg = binary_fill_holes(fg)

    fg = binary_dilation(fg, iterations=10)

    bg = ~fg
    mask_img = Image.fromarray((bg.astype(np.uint8) * 255))

    mask_img.save("bg_mask.png")

    coverage = np.sum(bg) / bg.size * 100
    print(f"[BG MASK] Coverage={coverage:.1f}% saved: {os.path.abspath('bg_mask.png')}")

    return os.path.abspath("bg_mask.png")

In [ ]:
@tool
def edit_image(image_path: str, mask_path: str, edit_prompt: str, edit_type: str = "auto") -> str:
    """
    Stable inpainting tool with:
    - Feathered masks (no hard edges)
    - Prompt conditioning
    - Adaptive strength (reduces randomness)
    - Structure-preserving generation
    """


    print(f"[EDIT] image='{image_path}', prompt='{edit_prompt}', type='{edit_type}'")

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(mask_path):
        raise FileNotFoundError(f"Mask not found: {mask_path}")

    if edit_type == "auto":
        edit_type = detect_edit_type(edit_prompt)
    print(f"[EDIT] Detected type: {edit_type}")

    img = Image.open(image_path).convert("RGB")
    raw_mask = Image.open(mask_path).convert("L")

    mask_np = np.array(raw_mask) / 255.0
    mask_np = gaussian_filter(mask_np, sigma=5)
    mask_np = np.clip(mask_np * 1.2, 0, 1)
    mask_img = Image.fromarray((mask_np * 255).astype(np.uint8))

    original_size = img.size
    size = (768, 768) if torch.cuda.is_available() else (512, 512)
    img_r = img.resize(size, Image.Resampling.LANCZOS)
    mask_r = mask_img.resize(size, Image.Resampling.LANCZOS)

    prompt, negative, strength, control_guidance, steps, cfg = build_prompt_and_params(edit_prompt, edit_type)

    if control_guidance > 0:
        image_np = np.array(img_r)
        line = cv2.ximgproc.createFastLineDetector().detect(image_np)
        control_image = Image.fromarray(line)
        result = inpaint_pipeline(
            prompt=prompt,
            negative_prompt=negative,
            image=img_r,
            mask_image=mask_r,
            num_inference_steps=steps,
            guidance_scale=cfg,
            strength=strength,
            height=size[1],
            width=size[0],
            generator=torch.Generator(device=device).manual_seed(42),
            control_image=control_image,
            controlnet_conditioning_scale=control_guidance
        )
    else:
        result = inpaint_pipeline(
            prompt=prompt,
            negative_prompt=negative,
            image=img_r,
            mask_image=mask_r,
            num_inference_steps=steps,
            guidance_scale=cfg,
            strength=strength,
            height=size[1],
            width=size[0],
            generator=torch.Generator(device=device).manual_seed(42)
        )

    out = result.images[0].resize(original_size, Image.Resampling.LANCZOS)

    os.makedirs("final_outputs", exist_ok=True)
    out_path = f"final_outputs/edited_{edit_type}_{os.path.basename(image_path)}"
    out.save(out_path)

    print(f"[EDIT] Saved: {out_path}")
    return os.path.abspath(out_path)

def detect_edit_type(edit_prompt: str) -> str:
    prompt_lower = edit_prompt.lower()

    removal_words = ['remove', 'delete', 'erase', 'take out', 'hide', 'eliminate']
    if any(word in prompt_lower for word in removal_words):
        return "remove"

    enhance_words = ['enhance', 'improve', 'fix', 'repair', 'clear', 'sharp', 'better quality']
    if any(word in prompt_lower for word in enhance_words):
        return "enhance"

    transform_words = ['style', 'filter', 'look like', 'painting', 'cartoon', 'fantasy', 'artistic', 'animated']
    if any(word in prompt_lower for word in transform_words):
        return "transform"

    replacement_patterns = [
        'background', 'sky', 'wall', 'floor', 'texture', 'replace', 'with',
        'instead of', 'swap', 'change', 'to', 'turn into', 'convert to',
        'make it', 'transform into'
    ]

    if any(word in prompt_lower for word in replacement_patterns):
        return "replace"

    if " to " in prompt_lower and len(prompt_lower.split()) <= 10:
        return "replace"

    return "modify"

def build_prompt_and_params(edit_prompt: str, edit_type: str):
    base_pos = "high quality, realistic, seamless, natural lighting, coherent texture, professional"
    base_neg = "blurry, distorted, artifacts, repeating patterns, deformed, low detail, watermark, text"

    if edit_type == "remove":
        return (
            f"{base_pos}, natural background continuation, clean fill, no traces, {edit_prompt}",
            f"{base_neg}, object, person, duplicate, patchy textures",
            1.0,
            0.0,
            40,
            7.5
        )

    if edit_type == "add":
        return (
            f"{base_pos}, {edit_prompt}, matches scene lighting, correct perspective, coherent shadows",
            f"{base_neg}, floating, bad perspective, inconsistent lighting",
            0.95,
            0.6,
            50,
            8.5
        )

    if edit_type == "replace":
        return (
            f"{edit_prompt}, realistic, same lighting, correct depth, seamless transition, professional photo",
            f"{base_neg}, fake texture, obvious patch, mismatch lighting",
            0.9,
            0.0,
            45,
            8.0
        )

    if edit_type == "enhance":
        return (
            f"{base_pos}, enhanced detail, sharper {edit_prompt}, high resolution",
            base_neg,
            0.5,
            0.0,
            35,
            7.0
        )

    if edit_type == "transform":
        return (
            f"{edit_prompt}, artistic, detailed, consistent subject identity",
            base_neg,
            0.75,
            0.8,
            45,
            9.0
        )

    return (
        f"{base_pos}, subtle {edit_prompt}, coherent colors, consistent style",
        f"{base_neg}, identity change, mismatch texture",
        0.55,
        0.5,
        40,
        8.0
    )

In [ ]:
tools = [segment_objects, segment_multiple, get_background_mask_path, edit_image]

llm = ChatGroq(
    temperature=0,
    model="llama-3.1-8b-instant"
)

prompt_template = """
You are an expert photo editor with one universal tool:

**edit_image(image_path, mask_path, edit_prompt)**

**SMART SEGMENTATION STRATEGY:**
- For object edits: segment_objects(image_path, "object_name")
- For background edits:
    1) Identify subjects (e.g., "person", "car", "main subject")
    2) Call: get_background_mask_path(image_path, ["person", "main subject"])
    3) Then call edit_image(image_path, bg_mask, "<replacement prompt>")
- For whole-image edits: segment_objects(image_path, "entire image")
- When unsure: try segment_multiple() to see available objects

**INTELLIGENT PROMPT CRAFTING:**
- Be specific: "change water to sandy beach" not just "change background"
- Use natural language: "remove the trash can" not "remove object"
- Include context: "forest background with trees" not just "forest"

**EXAMPLES:**
User: "change water to land"
→ get_background_mask_path("img.jpg", ["person", "main subject"]) → mask_path (for example - "/content/mask.png")
→ edit_image("img.jpg", mask_path, "change water to grassy land with trees")

User: "make the car red"
→ segment_objects("img.jpg", "car") → mask_path
→ edit_image("img.jpg", mask_path, "red car paint color")

User: "enhance the face"
→ segment_objects("img.jpg", "face") → mask_path
→ edit_image("img.jpg", mask_path, "enhance face details and skin texture")

The tool automatically optimizes parameters based on your natural language.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_template),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
user_request = "In the file '/content/cat.png', change background to beach."
result = agent_executor.invoke({"input": user_request})

print("\n--- Agent Run Complete ---")
print(f"Final output file path: {result['output']}")

In [ ]:
user_request = "In the file '/content/cat.png', make the cat animated."
result = agent_executor.invoke({"input": user_request})

print("\n--- Agent Run Complete ---")
print(f"Final output file path: {result['output']}")

In [ ]:
user_request = "In the file '/content/kuchbhi.png', make the person's beard black."
result = agent_executor.invoke({"input": user_request})

print("\n--- Agent Run Complete ---")
print(f"Final output file path: {result['output']}")
